### Import Credentials

In [0]:
%run './Config'

In [0]:
# Mounting Directory for Census CSV Intake
storageAccount = "gen10datafund2205"
storageContainer = "group1-marketing-analytics"
mount_point = "/mnt/group1/educationin"
    
    
configs2 = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret,
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point)
except:
    pass


dbutils.fs.mount(
source = f"abfss://{storageContainer}@{storageAccount}.dfs.core.windows.net/", 
mount_point = mount_point, 
extra_configs = configs2)

/mnt/group1/educationin has been unmounted.
Out[1]: True

In [0]:
from pyspark.sql.functions import col

# Read CSV. Set first row as header and remove the second row.
educ_data = spark.read.format('csv').options(header='True').load('/mnt/group1/educationin/ACSST5Y2020.S1501_data_with_overlays_2022-04-22T163802.csv')
educ_data = educ_data.filter(col("GEO_ID") != 'id')

In [0]:
# Select only the two needed columns and rename them. Show the results.
educ_data = educ_data.select("GEO_ID", "S1501_C02_012E")
educ_data = educ_data.withColumnRenamed("GEO_ID", "ZipCode")\
       .withColumnRenamed("S1501_C02_012E", "CensusEducationPct")
educ_data = educ_data.withColumn("ZipCode", educ_data.ZipCode.substr(10,14))

educ_data.show()

+-------+------------------+
ZipCode|CensusEducationPct|
+-------+------------------+
 00601| 13.8|
 00602| 15.5|
 00603| 16.3|
 00606| 7.0|
 00610| 13.5|
 00612| 18.7|
 00616| 17.2|
 00617| 17.9|
 00622| 14.6|
 00623| 20.5|
 00624| 18.7|
 00627| 15.9|
 00631| 13.9|
 00637| 16.5|
 00638| 12.2|
 00641| 16.1|
 00646| 20.9|
 00647| 16.4|
 00650| 12.2|
 00652| 23.0|
+-------+------------------+
only showing top 20 rows

In [0]:
# Write DataFrame to JSON file
educ_data.write.format('json').mode('overwrite').save(
    '/mnt/group1/educationin/education_cleaned.json')